##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 基于 TensorFlow Lite Model Maker 的音频域迁移学习

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/lite/models/modify/model_maker/audio_classification"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/lite/models/modify/model_maker/audio_classification.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/lite/models/modify/model_maker/audio_classification.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 Github 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/lite/models/modify/model_maker/audio_classification.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>
</td>
  <td>     <a href="https://tfhub.dev/google/yamnet/1"><img src="https://tensorflow.google.cn/images/hub_logo_32px.png">查看 TF Hub 模型 </a>
</td>
</table>

在此 CoLab 笔记本中，您将学习如何使用 [TensorFlow Lite Model Maker](https://tensorflow.google.cn/lite/models/modify/model_maker) 来训练自定义音频分类模型。

Model Maker 库能够使用迁移学习来简化使用自定义数据集训练 TensorFlow Lite 模型的过程。使用您自己的自定义数据集重新训练 TensorFlow Lite 模型可以减少所需的训练数据量，并将缩短训练时间。

这是[在 Android 上自定义并部署音频模型 Codelab](https://codelabs.developers.google.com/codelabs/tflite-audio-classification-custom-model-android) 中的一部分。

您将使用一个自定义的鸟类数据集，并导出一个可在手机上使用的 TFLite 模型、一个可用于在浏览器中进行推断的 TensorFlow.JS 模型，以及一个可用于服务的 SavedModel 版本。


## 安装依赖项


In [ ]:
!sudo apt -y install libportaudio2
!pip install tflite-model-maker

## 导入 TensorFlow、Model Maker 和其他库

在所需的依赖项中，您将使用 TensorFlow 和 Model Maker。除了这些，其他依赖项用于音频操作、播放和可视化。

In [ ]:
import tensorflow as tf
import tflite_model_maker as mm
from tflite_model_maker import audio_classifier
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import glob
import random

from IPython.display import Audio, Image
from scipy.io import wavfile

print(f"TensorFlow Version: {tf.__version__}")
print(f"Model Maker Version: {mm.__version__}")

## Birds 数据集

Birds 数据集是 5 种鸟类歌声的教育集合：

- White-breasted Wood-Wren（白胸林鹩）
- House Sparrow（家麻雀）
- Red Crossbill（红交嘴雀）
- Chestnut-crowned Antpitta（栗顶蚁鸫）
- Azara's Spinetail（阿氏针尾雀）

原始音频来自 [Xeno-canto](https://www.xeno-canto.org/)，这是一个致力于分享世界各地鸟鸣的网站。

我们从下载数据开始。

In [ ]:
birds_dataset_folder = tf.keras.utils.get_file('birds_dataset.zip',
                                                'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/birds_dataset.zip',
                                                cache_dir='./',
                                                cache_subdir='dataset',
                                                extract=True)
                                                

## 探索数据

音频已被拆分为训练文件夹和测试文件夹。在每个拆分的文件夹中，每种鸟都有一个文件夹，使用它们的 `bird_code` 作为文件名。

音频均为单声道，采样率为 16 kHz。

有关每个文件的详细信息，请阅读 `metadata.csv` 文件。其中包含所有文件的作者、链接和一些详细信息。在本教程中，您不需要自己阅读它。

In [ ]:
# @title [Run this] Util functions and data structures.

data_dir = './dataset/small_birds_dataset'

bird_code_to_name = {
  'wbwwre1': 'White-breasted Wood-Wren',
  'houspa': 'House Sparrow',
  'redcro': 'Red Crossbill',  
  'chcant2': 'Chestnut-crowned Antpitta',
  'azaspi1': "Azara's Spinetail",   
}

birds_images = {
  'wbwwre1': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/22/Henicorhina_leucosticta_%28Cucarachero_pechiblanco%29_-_Juvenil_%2814037225664%29.jpg/640px-Henicorhina_leucosticta_%28Cucarachero_pechiblanco%29_-_Juvenil_%2814037225664%29.jpg', # 	Alejandro Bayer Tamayo from Armenia, Colombia 
  'houspa': 'https://upload.wikimedia.org/wikipedia/commons/thumb/5/52/House_Sparrow%2C_England_-_May_09.jpg/571px-House_Sparrow%2C_England_-_May_09.jpg', # 	Diliff
  'redcro': 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/Red_Crossbills_%28Male%29.jpg/640px-Red_Crossbills_%28Male%29.jpg', #  Elaine R. Wilson, www.naturespicsonline.com
  'chcant2': 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/67/Chestnut-crowned_antpitta_%2846933264335%29.jpg/640px-Chestnut-crowned_antpitta_%2846933264335%29.jpg', # 	Mike's Birds from Riverside, CA, US
  'azaspi1': 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/Synallaxis_azarae_76608368.jpg/640px-Synallaxis_azarae_76608368.jpg', # https://www.inaturalist.org/photos/76608368
}

test_files = os.path.abspath(os.path.join(data_dir, 'test/*/*.wav'))

def get_random_audio_file():
  test_list = glob.glob(test_files)
  random_audio_path = random.choice(test_list)
  return random_audio_path


def show_bird_data(audio_path):
  sample_rate, audio_data = wavfile.read(audio_path, 'rb')

  bird_code = audio_path.split('/')[-2]
  print(f'Bird name: {bird_code_to_name[bird_code]}')
  print(f'Bird code: {bird_code}')
  display(Image(birds_images[bird_code]))

  plttitle = f'{bird_code_to_name[bird_code]} ({bird_code})'
  plt.title(plttitle)
  plt.plot(audio_data)
  display(Audio(audio_data, rate=sample_rate))

print('functions and data structures created')

### 播放一些音频

为了更好地理解数据，我们来听一听测试拆分中的随机音频文件。

注：在本笔记本的后面部分，您将对此音频运行推断以进行测试

In [ ]:
random_audio = get_random_audio_file()
show_bird_data(random_audio)

## 训练模型

使用 Model Maker 制作音频时，必须从模型规范开始。这是基本模型，您的新模型将从中提取信息以学习新类。它还会影响如何转换数据集以符合模型规范参数，例如：采样率、通道数。

[YAMNet](https://tfhub.dev/google/yamnet/1) 是在 AudioSet 数据集上训练的音频事件分类器，用于从 AudioSet 本体预测音频事件。

它的输入频率预计为 16 kHz，具有 1 个通道。

您无需自己进行任何重采样。Model Maker 会为您完成。

- `frame_length` 用于确定每个训练样本的长度。在此示例中为 EXPECTED_WAVEFORM_LENGTH * 3s

- `frame_steps` 用于确定训练样本之间的距离。在本例中，第 i 个样本将在第 (i-1) 个样本后的 EXPECTED_WAVEFORM_LENGTH * 6s 处开始。

设置这些值的原因是为了绕过现实世界数据集中的一些限制。

例如，在鸟类数据集中，鸟类并不总是唱歌。它们会唱歌，休息，然后再唱歌，中间会有噪音。拥有较长的帧将有助于捕捉歌声，但将其设置得太长会减少用于训练的样本数量。


In [ ]:
spec = audio_classifier.YamNetSpec(
    keep_yamnet_and_custom_heads=True,
    frame_step=3 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH,
    frame_length=6 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH)

## 加载数据

Model Maker 具有从文件夹加载数据并以模型规范的预期格式提供数据的 API。

训练拆分和测试拆分基于文件夹。验证数据集将被创建为训练拆分的 20%。

注：`cache=True` 对于提高之后的训练速度很重要，但它也需要更多的 RAM 来保存数据。对于 Birds 数据集，这不是问题，因为它只有 300MB，但如果您使用自己的数据，则必须加以注意。


In [ ]:
train_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'train'), cache=True)
train_data, validation_data = train_data.split(0.8)
test_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'test'), cache=True)

## 训练模型

audio_classifier 具有 [`create`](https://tensorflow.google.cn/lite/api_docs/python/tflite_model_maker/audio_classifier/create) 方法，用于创建并开始训练模型。

您可以自定义许多参数，有关更多信息，请阅读文档中的更多详细信息。

在第一次尝试中，您将使用所有默认配置并训练 100 个周期。

注：第一个周期会比所有其他周期花费更长的时间，因为此时会创建缓存。之后，每一个周期花费近 1 秒。

In [ ]:
batch_size = 128
epochs = 100

print('Training the model')
model = audio_classifier.create(
    train_data,
    spec,
    validation_data,
    batch_size=batch_size,
    epochs=epochs)

准确率看起来很好，但重要的是对测试数据运行评估步骤，并验证您的模型是否能够在非种子数据上取得良好的结果。

In [ ]:
print('Evaluating the model')
model.evaluate(test_data)

## 理解模型

训练分类器时，查看[混淆矩阵](https://en.wikipedia.org/wiki/Confusion_matrix)非常实用。混淆矩阵可帮助您详细了解分类器在测试数据上的性能。

Model Maker 已经为您创建了混淆矩阵。

In [ ]:
def show_confusion_matrix(confusion, test_labels):
  """Compute confusion matrix and normalize."""
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  axis_labels = test_labels
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

confusion_matrix = model.confusion_matrix(test_data)
show_confusion_matrix(confusion_matrix.numpy(), test_data.index_to_label)

## 测试模型 [可选]

您可以使用测试数据集中的样本音频试用该模型，以查看结果。

首先，您获得应用模型。

In [ ]:
serving_model = model.create_serving_model()

print(f'Model\'s input shape and type: {serving_model.inputs}')
print(f'Model\'s output shape and type: {serving_model.outputs}')

回到您之前加载的随机音频

In [ ]:
# if you want to try another file just uncoment the line below
random_audio = get_random_audio_file()
show_bird_data(random_audio)

创建的模型具有固定的输入窗口。

对于给定的音频文件，您必须将其拆分成预期大小的数据窗口。最后一个窗口可能需要用零填充。

In [ ]:
sample_rate, audio_data = wavfile.read(random_audio, 'rb')

audio_data = np.array(audio_data) / tf.int16.max
input_size = serving_model.input_shape[1]

splitted_audio_data = tf.signal.frame(audio_data, input_size, input_size, pad_end=True, pad_value=0)

print(f'Test audio path: {random_audio}')
print(f'Original size of the audio data: {len(audio_data)}')
print(f'Number of windows for inference: {len(splitted_audio_data)}')

您将循环遍历所有拆分的音频，并为每个音频应用模型。

您刚刚训练的模型有两个输出：原始 YAMNet 的输出和您刚刚训练的输出。这一点很重要，因为现实世界的环境比鸟鸣要复杂得多。您可以使用 YAMNet 的输出过滤掉不相关的音频，例如，在鸟类用例中，如果 YAMNet 没有对 Birds 或 Animals 进行分类，这可能表明您的模型的输出可能具有不相关的分类。

下面打印了两个输出，以便于理解它们之间的关系。您的模型犯错的大多数时候是当 YAMNet 的预测与您的领域不相关时（例如：鸟类）。

In [ ]:
print(random_audio)

results = []
print('Result of the window ith:  your model class -> score,  (spec class -> score)')
for i, data in enumerate(splitted_audio_data):
  yamnet_output, inference = serving_model(data)
  results.append(inference[0].numpy())
  result_index = tf.argmax(inference[0])
  spec_result_index = tf.argmax(yamnet_output[0])
  t = spec._yamnet_labels()[spec_result_index]
  result_str = f'Result of the window {i}: ' \
  f'\t{test_data.index_to_label[result_index]} -> {inference[0][result_index].numpy():.3f}, ' \
  f'\t({spec._yamnet_labels()[spec_result_index]} -> {yamnet_output[0][spec_result_index]:.3f})'
  print(result_str)


results_np = np.array(results)
mean_results = results_np.mean(axis=0)
result_index = mean_results.argmax()
print(f'Mean result: {test_data.index_to_label[result_index]} -> {mean_results[result_index]}')

## 导出模型

最后一步是导出要在嵌入式设备或浏览器上使用的模型。

`export` 方法能够为您导出这两种格式。

In [ ]:
models_path = './birds_models'
print(f'Exporing the TFLite model to {models_path}')

model.export(models_path, tflite_filename='my_birds_model.tflite')

您还可以导出 SavedModel 版本，以便在 Python 环境中应用或使用。

In [ ]:
model.export(models_path, export_format=[mm.ExportFormat.SAVED_MODEL, mm.ExportFormat.LABEL])

## 后续步骤

您成功了。

现在，您的新模型可以使用 [TFLite AudioClassifier Task API](https://tensorflow.google.cn/lite/inference_with_metadata/task_library/audio_classifier) 部署在移动设备上。

您还可以使用具有不同类的您自己的数据尝试相同的过程，这里是[用于音频分类的 Model Maker](https://tensorflow.google.cn/lite/api_docs/python/tflite_model_maker/audio_classifier) 的文档。

您还可以从端到端参考应用中学习：[Android](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/android/)，[iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/ios)。